In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from scipy import stats
from scipy.stats import kendalltau
%matplotlib inline

In [ ]:
data = pd.read_csv('/kaggle/input/house-prices-advanced-regression-techniques/train.csv')

In [ ]:
data.head()

In [ ]:
data.info()

LotFrontage ,Alley , MasVnrType, MasVnrArea, BsmtQual , BsmtCond , BsmtExposure , BsmtFinType1,   BsmtFinType2, Electrical,  FireplaceQu ,  GarageType , GarageYrBlt ,GarageFinish , GarageQual , GarageCond ,  PoolQC , Fence , MiscFeature have missing values. In these Alley, PoolQC, Fence and MiscFeature can be removed as too less info.


In [ ]:
data = data.drop(['Alley','PoolQC','Fence','MiscFeature'], axis=1)

In [ ]:
plt.figure(figsize = (40,40))
sns.heatmap(data.drop(['Id','SalePrice'], axis=1).corr(), annot =True);
#data.drop(['Id','SalePrice'],axis =1).corr()

Checking for multicollinearity:
if pearson correlation is above 0.7, then we confirm multicollinearity:
From the heatmap, the following correlation can be identified:
1. YearBuilt and GarageYrBlt
2. TotalBsmtSF and 1stFlrSF
3. GrLivArea and TotRmsAbvGrd
4. GarageCars and GarageArea


In [ ]:
plt.scatter(x='YearBuilt', y='SalePrice', data=data, alpha =0.05)
plt.scatter(x='GarageYrBlt', y='SalePrice', data=data, alpha=0.5)
plt.show()
# shows the correlation
data.drop('GarageYrBlt', axis =1, inplace=True)

In [ ]:
plt.scatter(x='TotalBsmtSF', y='SalePrice', data=data, alpha =0.5)
plt.scatter(x='1stFlrSF', y='SalePrice', data=data, alpha=0.5)
plt.show()
# shows the correlation
data.drop('1stFlrSF', axis =1, inplace=True)

In [ ]:
plt.scatter(x='GrLivArea', y='SalePrice', data=data, alpha =0.5, label='GR')
plt.scatter(x='TotRmsAbvGrd', y='SalePrice', data=data, alpha=0.5, label='Tot')
plt.legend()
plt.show()
print(kendalltau(data['GrLivArea'],data['TotRmsAbvGrd']))
# shows the correlation, CONFUSED
plt.scatter(x='GarageCars', y='SalePrice', data=data, alpha =0.5)
plt.scatter(x='GarageArea', y='SalePrice', data=data, alpha=0.5)
plt.show() 
print(kendalltau(data['GarageCars'],data['GarageArea']))
# These two graph brings up a new issue ie  does pearson correlation works when one variable is ordinal and other numerical.
# kendall tau correlation was suggested
data.drop(['TotRmsAbvGrd','GarageCars'],axis=1, inplace=True)
##    https://journals.sagepub.com/doi/pdf/10.1177/8756479308317006
##    https://machinelearningmastery.com/how-to-calculate-nonparametric-rank-correlation-in-python/

In [ ]:
data['FireplaceQu'].unique()
# Shows that values can be converted to ordinal.

In [ ]:
data['FireplaceQu']=data['FireplaceQu'].map({'Ex':5,'Gd':4, 'TA':3, 'Fa':2, 'Po':1})
data['FireplaceQu'].value_counts()
sns.boxplot(x='FireplaceQu', y='SalePrice', data=data)


there is a linear relation with fireplace quality, the better the costlier, but lot of outliers also there for '3' output and lot of NaN values too

In [ ]:
plt.figure(figsize = (7.5,7.5))
sns.distplot(data['SalePrice']);

In [ ]:
print(stats.skew(data['SalePrice']))
print(stats.kurtosis(data['SalePrice']))


The sale price shows following issues:
1. positive skewness
2. is it Bimodal ??
3. Peakedness (kurtosis)
4. hence not a normal distribution

In [ ]:
data.columns

Assumptions made on Simple Linear Regression system:
1. The relation is linear
2. Multivariate normality
3. No Multicollinearity
4. No Autocorrelation (Durbin-Watson)
5. Homoscedasticity

To check for linear relation, what we could is to make  scatter plots between all numerical variables, and check for linear relation.
numerical variables are:
LotFrontage
LotArea
YearBuilt
YearRemodAdd
BsmtFinSF1
BsmtFinSF2
TotalBsmtSF
1stFlrSF
2ndFlrSF
LowQualFinSF
GrLivArea
BsmtFullBath
BsmtHalfBath
FullBath
HalfBath
BedroomAbvGr
KitchenAbvGr
TotRmsAbvGrd
Fireplaces
GarageYrBlt
GarageCars
GarageArea
WoodDeckSF
OpenPorchSF
EnclosedPorch
3SsnPorch
ScreenPorch
PoolArea

In [ ]:
sns.scatterplot(x= data['LotFrontage'], y=data['SalePrice'], alpha=0.5);


In [ ]:
def scatter_plot(l, p=20, g=6):
    fig, ax = plt.subplot(len(l.columns), figsize=(p,g))
    for i in range(len(l.columns)):
        ax[i].scatter(x = l[:,i])
    plt.show()

In [ ]:
lmao = data[["LotFrontage","LotArea","YearBuilt","YearRemodAdd","BsmtFinSF1","BsmtFinSF2","TotalBsmtSF","2ndFlrSF","LowQualFinSF",
             "GrLivArea","BsmtFullBath","BsmtHalfBath","FullBath","HalfBath","BedroomAbvGr","KitchenAbvGr","Fireplaces"
            ,"GarageArea","WoodDeckSF","OpenPorchSF","EnclosedPorch","3SsnPorch","ScreenPorch","PoolArea"]]

In [ ]:
lmao.hist(figsize=(20,20));